# 最后的流程

# 重新处理数据

## 多CAR

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer

# 1. 读取数据
df = pd.read_excel('data_guba.xlsx')

# 2. 提取异常收益列
ar_cols = [col for col in df.columns if col.startswith('AR_')]
id_vars = [col for col in df.columns if col not in ar_cols]

# 3. 宽转长
df_long = pd.melt(df, id_vars=id_vars, value_vars=ar_cols,
                  var_name='time', value_name='AR')
df_long['time_num'] = df_long['time'].str.replace('AR_', '').astype(int)

# 4. 缺失值处理
imputer = SimpleImputer(strategy='mean')
df_long[['AR']] = imputer.fit_transform(df_long[['AR']])

# 6. 重命名
rename_dict = {
    '年报资产报酬率（%）': 'AssetReturn',
    '年报资产报酬率TTM（%）': 'AssetReturn_TTM',
    '年报营业利润率（%）': 'OperatingProfitMargin',
    '流动比率（%）': 'CurrentRatio',
    '速动比率（%）': 'QuickRatio',
    '净资产负债率（%）': 'NetDebtRatio',
    '年报总资产周转率（次）': 'AssetTurnover',
    '年报总资产周转率TTM（次）': 'AssetTurnover_TTM',
    '资产负债率（%）': 'DebtRatio',
    '年报非流动资产/总资产（%）': 'NonCurrentAsset_Ratio'
}
df_long.rename(columns=rename_dict, inplace=True)

# 7. 控制变量列表
control_vars = [
    'AssetReturn', 'AssetReturn_TTM', 'OperatingProfitMargin',
    'CurrentRatio', 'QuickRatio', 'NetDebtRatio',
    'AssetTurnover', 'AssetTurnover_TTM', 'DebtRatio',
    'NonCurrentAsset_Ratio'
]

# 8. 统一id_vars
id_vars = control_vars + ['is_authoritative']

# 9. 定义计算CAR的函数
def calculate_car(df_long, start, end, id_vars=id_vars):
    df_car = df_long[df_long['time_num'].between(start, end)]
    df_car = df_car.groupby(id_vars)['AR'].sum().reset_index()
    df_car.rename(columns={'AR': f'CAR_{start}_{end}'}, inplace=True)
    return df_car

# 10. 计算多个时间窗的CAR
car_windows = [(-1, 3), (-1, 1), (-1, 2), (-1,4), (-1,5)]
car_dfs = [calculate_car(df_long, start, end) for (start, end) in car_windows]

# 11. 合并所有CAR结果
from functools import reduce
df_car_merged = reduce(lambda left, right: pd.merge(left, right, on=id_vars, how='outer'), car_dfs)

# 12. 合并回原始数据（删除重复列）
df_final = df_long.drop(columns=['time', 'time_num']).drop_duplicates(subset=id_vars)
df_final = df_final.merge(df_car_merged, on=id_vars, how='left')

# 13. 保存到Excel：每个CAR时间窗一列
df_final.to_excel('CAR_results_combined.xlsx', index=False)

# ✅ 额外（如需按sheet保存）：
with pd.ExcelWriter('CAR_results_by_window.xlsx', engine='openpyxl') as writer:
    for (start, end), df_car in zip(car_windows, car_dfs):
        df_car.to_excel(writer, sheet_name=f'CAR_{start}_{end}', index=False)


## 新控制变量 

### 提取年份

In [1]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel("CAR_results_combined.xlsx")

# 确保 event date 列是 datetime 类型
df["event date"] = pd.to_datetime(df["event date"])

# 提取年份并创建新列 year
df["year"] = df["event date"].dt.year

# 查看结果
print(df.head())

# 如果需要保存回 Excel
df.to_excel("CAR_results_combined.xlsx", index=False)


     股票代码 event date     交易日0_x  市场类型_x     alpha      beta  AssetReturn  \
0  600141 2015-01-02 2015-01-06       1  0.000835  0.124236       3.9631   
1     156 2015-01-04 2015-01-05       4  0.000088  0.899838       5.6577   
2    2037 2015-01-04 2015-01-05       4  0.002651  0.579189       4.0469   
3    2055 2015-01-07 2015-01-07       4 -0.000427  0.754313       3.1448   
4    2030 2015-01-07 2015-01-07       4  0.000185  1.327853       6.7978   

   AssetReturn_TTM  OperatingProfitMargin  CurrentRatio  ...          mean  \
0           0.4989                 1.2332        0.4475  ...  1.333333e-01   
1           5.8418                17.8081        3.4528  ... -2.222222e-02   
2           1.1130                 3.4687        0.8489  ...  1.666667e-01   
3           1.3823                 1.9199        1.1927  ... -5.551115e-18   
4           5.0844                 5.9033        2.1392  ...  9.800000e-02   

   variance  is_authoritative        AR  CAR_-1_3  CAR_-1_1  CAR_-1_2  \
0

### 归并AR

In [6]:
import pandas as pd

df1 = pd.read_excel('CAR_results_combined.xlsx')
df2 = pd.read_excel("data_guba.xlsx")

# 2. 如果需要，统一 event date 的数据类型（例如转换为日期类型）
df1['股票代码'] = df1['股票代码'].astype(str)
df2['股票代码'] = df2['股票代码'].astype(str)

df1['event date'] = pd.to_datetime(df1['event date'])
df2['event date'] = df2['event date'].astype(str).str.strip()
df2['event date'] = df2['event date'].str.split().str[0]
df2['event date'] = pd.to_datetime(df2['event date'])

# 3. 根据共同键（股票代码, event date）进行合并
# 这里采用 inner join：只保留两个表中都存在的匹配行
merged_df = pd.merge(df1, df2, on=['股票代码', 'event date'], how='left')

# 4. 将合并后的结果保存到新的 Excel 文件中
merged_df.to_excel('CAR_results_combined.xlsx', index=False)

print("两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx")

两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx


### 合并新控制变量

In [7]:
import pandas as pd

df1 = pd.read_excel('CAR_results_combined.xlsx')
df2 = pd.read_excel("control_vars.xlsx")

# 2. 如果需要，统一 event date 的数据类型（例如转换为日期类型）
df1['股票代码'] = df1['股票代码'].astype(str)
df2['股票代码'] = df2['股票代码'].astype(str)


# 3. 根据共同键（股票代码, event date）进行合并
# 这里采用 inner join：只保留两个表中都存在的匹配行
merged_df = pd.merge(df1, df2, on=['股票代码', 'year'], how='left')

# 4. 将合并后的结果保存到新的 Excel 文件中
merged_df.to_excel('last_step.xlsx', index=False)

print("两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx")

两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx


### 合并姜富伟

In [5]:
import pandas as pd

df1 = pd.read_excel('last_step.xlsx')
df2 = pd.read_excel("fuwei.xlsx")

# 2. 如果需要，统一 event date 的数据类型（例如转换为日期类型）
df1['股票代码'] = df1['股票代码'].astype(str)
df2['股票代码'] = df2['股票代码'].astype(str)
df1['event date'] = pd.to_datetime(df1['event date'])
df2['event date'] = pd.to_datetime(df2['event date'])

# 3. 根据共同键（股票代码, event date）进行合并
# 这里采用 inner join：只保留两个表中都存在的匹配行
merged_df = pd.merge(df1, df2, on=['股票代码', 'event date'], how='left')

# 4. 将合并后的结果保存到新的 Excel 文件中
merged_df.to_excel('last_step.xlsx', index=False)

print("两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx")

两个 Excel 文件已成功根据 (股票代码, event date) 合并，并保存到 merged_output.xlsx


## 计算新控制变量

In [8]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel("last_step.xlsx")

# 
df['Ind'] = df['行业代码'].astype('category').cat.codes

df['Firm'] = df['股票代码'].astype('category').cat.codes

# 如果需要保存回 Excel
df.to_excel("last_step.xlsx", index=False)

# 描述性统计

In [9]:
import pandas as pd

# 1. 读取 Excel 文件（请替换路径）
df = pd.read_excel('last_step.xlsx')

# 2. 指定你要统计的列（可手动列出，或用正则/前缀匹配）
columns_to_describe = ['ARE','OPM','CR','QR','ND','AT','sentiment','CAR_-1_3','TR','Ind','Soe','Dual','Indep','Lev','Size','Big4']  

# 3. 计算描述性统计
desc_stats = df[columns_to_describe].agg(['mean', 'median', 'std', 'min', 'max']).T
desc_stats.columns = ['均值', '中位数', '标准差', '最小值', '最大值']

# 4. 将结果保存为文本文件
with open('descriptive_statistics.txt', 'w', encoding='utf-8') as f:
    f.write(desc_stats.to_string())

print("描述性统计已保存到 descriptive_statistics.txt")


描述性统计已保存到 descriptive_statistics.txt


# 基准回归

In [4]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 读取数据（假设数据存储在 data.csv）
df = pd.read_excel("last_step.xlsx")
add_control_vars = True  # 是否添加控制变量
# 定义变量
dependent_var = "CAR_-1_3"  # 被解释变量
independent_var = "sentiment"  # 解释变量
control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']  # 控制变量

# 添加固定效应（行业 Ind，假设是分类变量）
df = pd.get_dummies(df, columns=['year'], drop_first=True)  # One-hot 编码行业固定效应

# 构建回归模型
if add_control_vars:
    # 如果有控制变量，添加到模型中
    X = df[[independent_var] + control_vars ]  # 解释变量+控制变量+行业固定效应
    
else:
    X = df[[independent_var]]  # 解释变量

X = sm.add_constant(X)  # 添加常数项
y = df[dependent_var]  # 被解释变量

X = X.replace([np.inf, -np.inf], np.nan).dropna()
y = y.loc[X.index]  # 确保 y 也同步删除相应行


# 运行 OLS 回归
model = sm.OLS(y, X).fit()

# 设置小数位数
decimal_places = 5  # 你可以自由修改这个值

# 获取回归结果，并设置小数位数
coefficients = model.params.round(decimal_places)  # 回归系数
p_values = model.pvalues.round(decimal_places)  # p 值
std_errors = model.bse.round(decimal_places)  # 标准误差
r_squared = round(model.rsquared, decimal_places)  # R²
adj_r_squared = round(model.rsquared_adj, decimal_places)  # 调整 R²

# 组织输出内容
output_text = []
output_text.append("回归结果（OLS 估计）\n")
output_text.append(f"R²: {r_squared},  调整 R²: {adj_r_squared}\n")
output_text.append("\n回归系数（Coefficient）：\n")
output_text.append(coefficients.to_string())
output_text.append("\n\n标准误差（Standard Error）：\n")
output_text.append(std_errors.to_string())
output_text.append("\n\nP 值（P-Values）：\n")
output_text.append(p_values.to_string())

# 写入 txt 文件
with open("regression_results.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(output_text))

print("回归结果已保存到 regression_results.txt")

回归结果已保存到 regression_results.txt


# 修订：新闻发布日期与临近交易日

In [2]:
import pandas as pd
import statsmodels.formula.api as smf

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')

# 2. 构造“不一致”变量：当“交易日0”与“event date”不一致时，mismatch=1，否则为0
df['mismatch'] = (df['交易日0'] != df['event date']).astype(int)

# 3. 计算事件窗 (-1, 1) 的累计异常收益 CAR = AR_-1 + AR_0 + AR_1
df['CAR'] = df['AR_-1'] + df['AR_0'] + df['AR_1']
control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']


# 6. 构造回归公式
formula = 'CAR ~ mismatch'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)

print("回归公式：", formula)

# 7. 使用 OLS 建模
model = smf.ols(formula, data=df).fit()
summary_text = model.summary().as_text()
with open('moderation_OLS_summary.txt', 'w') as f:
    f.write(summary_text)

回归公式： CAR ~ mismatch + ARE + OPM + CR + QR + ND + AT + Soe + Dual + Indep + Lev + Size + Big4


# 修订：DID

In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')

# 2. 提取所有以 "AR_" 开头的列作为事件窗口的异常收益
ar_cols = [col for col in df.columns if col.startswith('AR_')]

# 3. 确定 id_vars：保留除异常收益以外的列（假设已删去股票代码、event date、交易日0、市场类型等变量）
id_vars = [col for col in df.columns if col not in ar_cols]

# 4. 将数据从宽格式转换为长格式，生成新列 "time"（事件时点）和 "AR"（异常收益）
df_long = pd.melt(df, id_vars=id_vars, value_vars=ar_cols, var_name='time', value_name='AR')

# 5. 将 “time” 列转换为数值型变量
#    假设 time 格式为 "AR_-10", "AR_0", "AR_10" 等
df_long['time_num'] = df_long['time'].str.replace('AR_', '').astype(int)

# 6. 对 AR 列进行均值插补，处理缺失值
imputer = SimpleImputer(strategy='mean')
df_long[['AR']] = imputer.fit_transform(df_long[['AR']])

# 7. 构造处理后时点的虚拟变量：当 time_num >= 0 时取 1，否则取 0
df_long['Post'] = (df_long['time_num'] >= 0).astype(int)

# 8. 构造 DID 交互项：did = truth * Post
#    这里假设 truth 已经为 0/1 变量（1 表示受处理组，0 表示对照组）
df_long['did'] = df_long['truth'] * df_long['Post']
control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']
formula = 'AR ~ truth + Post + did'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)

print("回归公式：", formula)

# 12. 使用 OLS 估计 DID 模型（不采用分组聚类稳健标准误）
model = smf.ols(formula, data=df_long).fit()

# 使用 StringIO 捕获回归摘要输出
summary_text = model.summary().as_text()

# 将回归摘要写入文件
with open('model_summary.txt', 'w') as f:
    f.write(summary_text)


回归公式： AR ~ truth + Post + did + ARE + OPM + CR + QR + ND + AT + Soe + Dual + Indep + Lev + Size + Big4


# 解释变量的稳健性

In [12]:
import pandas as pd
import statsmodels.formula.api as smf

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')


control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4','truth']


# 6. 构造回归公式
formula = 'Q("CAR_-1_3") ~ Q("final_score")'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)

print("回归公式：", formula)

# 7. 使用 OLS 建模
model = smf.ols(formula, data=df).fit()
summary_text = model.summary().as_text()
with open('moderation_OLS_summary.txt', 'w') as f:
    f.write(summary_text)

回归公式： Q("CAR_-1_3") ~ Q("final_score") + ARE + OPM + CR + QR + ND + AT + Soe + Dual + Indep + Lev + Size + Big4 + truth


# 被解释变量的稳健性

In [8]:
import pandas as pd
import statsmodels.formula.api as smf

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')


control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']


# 6. 构造回归公式
formula = 'Q("CAR_-1_5") ~ sentiment'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)

print("回归公式：", formula)

# 7. 使用 OLS 建模
model = smf.ols(formula, data=df).fit()
summary_text = model.summary().as_text()
with open('moderation_OLS_summary.txt', 'w') as f:
    f.write(summary_text)

回归公式： Q("CAR_-1_5") ~ sentiment + ARE + OPM + CR + QR + ND + AT + Soe + Dual + Indep + Lev + Size + Big4


# 修正：调节效应

## 直接回归

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')


control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']


# 6. 构造回归公式
formula = 'Q("CAR_-1_3") ~ truth+ is_authoritative + truth : is_authoritative'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)

print("回归公式：", formula)

# 7. 使用 OLS 建模
model = smf.ols(formula, data=df).fit()
summary_text = model.summary().as_text()
with open('moderation_OLS_summary.txt', 'w') as f:
    f.write(summary_text)

回归公式： Q("CAR_-1_3") ~ truth+ is_authoritative + truth : is_authoritative


## 分组DID

In [17]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer

# 1. 读取数据（请将 'your_data.csv' 替换为你的数据文件路径）
df = pd.read_excel('last_step.xlsx')

# 2. 提取所有以 "AR_" 开头的列作为事件窗口的异常收益
ar_cols = [col for col in df.columns if col.startswith('AR_')]

# 3. 确定 id_vars：保留除异常收益以外的列（假设已删去股票代码、event date、交易日0、市场类型等变量）
id_vars = [col for col in df.columns if col not in ar_cols]

# 4. 将数据从宽格式转换为长格式，生成新列 "time"（事件时点）和 "AR"（异常收益）
df_long = pd.melt(df, id_vars=id_vars, value_vars=ar_cols, var_name='time', value_name='AR')

# 5. 将 “time” 列转换为数值型变量
#    假设 time 格式为 "AR_-10", "AR_0", "AR_10" 等
df_long['time_num'] = df_long['time'].str.replace('AR_', '').astype(int)

# 6. 对 AR 列进行均值插补，处理缺失值
imputer = SimpleImputer(strategy='mean')
df_long[['AR']] = imputer.fit_transform(df_long[['AR']])

# 7. 构造处理后时点的虚拟变量：当 time_num >= 0 时取 1，否则取 0
df_long['Post'] = (df_long['time_num'] >= 0).astype(int)

# 8. 构造 DID 交互项：did = truth * Post
#    这里假设 truth 已经为 0/1 变量（1 表示受处理组，0 表示对照组）
df_long['did'] = df_long['truth'] * df_long['Post']
control_vars = ['ARE', 'OPM', 'CR', 'QR', 'ND', 'AT', 'Soe', 'Dual', 'Indep', 'Lev', 'Size', 'Big4']


# 6. 构造回归公式
formula = 'AR ~ truth+ Post + did'
if control_vars:
    formula += ' + ' + ' + '.join(control_vars)
df_auth = df_long[df_long['is_authoritative'] == 1]
df_non_auth = df_long[df_long['is_authoritative'] == 0]

# 针对权威来源组运行回归
model_auth = smf.ols(formula, data=df_auth).fit()
summary_auth = model_auth.summary().as_text()

# 针对非权威来源组运行回归
model_non_auth = smf.ols(formula, data=df_non_auth).fit()
summary_non_auth = model_non_auth.summary().as_text()

# 将两个回归摘要保存到不同的文件中（也可以保存到同一个文件）
with open('model_summary_authoritative.txt', 'w') as f:
    f.write("DID Regression Results for is_authoritative = 1 (Authoritative Sources):\n")
    f.write(summary_auth)

with open('model_summary_non_authoritative.txt', 'w') as f:
    f.write("DID Regression Results for is_authoritative = 0 (Non-authoritative Sources):\n")
    f.write(summary_non_auth)